In [2]:
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.svm import LinearSVC

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [3]:
X = pd.read_csv('../datasets/algerian_formula_a.csv')
X

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,...,cos(DMC+DC),cos(DMC+ISI),cos(DMC+BUI),cos(DMC+FWI),cos(DC+ISI),cos(DC+BUI),cos(DC+FWI),cos(ISI+BUI),cos(ISI+FWI),cos(BUI+FWI)
0,1,6,2012,29,57,18,0.0,65.7,3.4,7.6,...,0.004426,-0.012389,0.869397,-0.725932,-0.865435,0.004426,-0.243544,-0.012389,-0.227202,-0.725932
1,2,6,2012,29,61,13,1.3,64.4,4.1,7.6,...,0.647596,0.377978,-0.145500,-0.210796,-0.678720,0.483305,-0.145500,0.186512,0.169967,-0.400799
2,3,6,2012,26,82,22,13.1,47.1,2.5,7.1,...,-0.984688,-0.942222,0.468517,-0.856889,0.438547,-0.930426,0.608351,-0.989992,0.921061,-0.942222
3,4,6,2012,25,89,13,2.5,28.6,1.3,6.9,...,-0.339155,0.267499,-0.989992,0.267499,0.815725,-0.678720,0.815725,-0.128844,1.000000,-0.128844
4,5,6,2012,27,77,16,0.0,64.8,3.0,14.2,...,-0.078678,-0.490261,0.815725,-0.936457,-0.952953,0.731991,-0.533584,0.377978,-0.128844,-0.307333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,26,9,2012,30,65,14,0.0,85.4,16.0,44.5,...,-0.689687,-0.079564,0.086614,-0.873305,0.300593,0.138497,0.742154,-0.830301,0.004426,-0.161238
240,27,9,2012,28,87,15,4.4,41.1,6.5,8.0,...,-0.354924,0.950233,0.991085,0.976588,-0.243544,-0.062792,-0.145500,0.999859,0.995004,0.996542
241,28,9,2012,27,87,29,0.5,45.9,3.5,7.9,...,0.393491,-0.725932,0.815725,-0.848100,-0.431377,0.299745,-0.243544,-0.790968,0.825336,-0.896758
242,29,9,2012,24,54,18,0.1,79.7,4.3,15.2,...,0.795815,0.960170,-0.999693,0.283662,-0.369768,0.120062,-0.981618,0.869397,-0.737394,0.885520


In [7]:
# df = pd.read_csv('../datasets/winequality-white.csv', sep=';')
df = pd.read_csv('../datasets/Algerian_forest_fires.csv')
# df = pd.read_csv("https://raw.githubusercontent.com/SahilSinhaLpu/Machine-Learning/master/Datasets/SomvervilleHappines.csv")
df

### Speaker DS
# Y = df['language'] 

### Algerian DS
Y = df['Classes']

### Banknote DS
# Y = df['Class']

### User knowledge DS
# Y = df['UNS']

## Wine quality DS
# Y = df['quality']

### Somerville DS
# Y = df['D']

## Pima Indians Diabetes DS
# Y = df['Outcome']
# Y
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

### KNN

In [144]:
knn = KNeighborsClassifier()

plain_sbs = SFS(knn, 
          k_features=(1, 10), 
          forward=False,
          floating=False, 
          scoring='accuracy',
          cv=10,
          n_jobs=-1)

plain_sbs.fit(X_train, y_train, custom_feature_names=X.columns)

selected_features = X.columns[list(plain_sbs.k_feature_idx_)]

x_t = X_train[selected_features]
clf = KNeighborsClassifier()
clf.fit(x_t, y_train)
print(cross_val_score(clf, x_t, y_train, cv=10).mean())

0.5127577117803643


### Naive Bayes

In [145]:
# Sequential Backward Selection
gnb = GaussianNB()

nb_sbs = SFS(gnb, 
          k_features=(1, 10), 
          forward=False, 
          floating=False, 
          scoring='accuracy',
          cv=10)

nb_sbs.fit(X_train, y_train, custom_feature_names=X.columns)

SequentialFeatureSelector(cv=10, estimator=GaussianNB(), forward=False,
                          k_features=(1, 10), scoring='accuracy')

In [146]:
selected_features = X.columns[list(nb_sbs.k_feature_idx_)]
x_nb = X_train[selected_features]

gnb_ = GaussianNB()
gnb_.fit(x_nb, y_train)
print(cross_val_score(gnb_, x_nb, y_train, cv=10).mean())

0.5216875880787099


### SVM

In [147]:
# Sequential Backward Selection
svm_sbs = SFS(svm.SVC(), 
          k_features=(1, 10), 
          forward=False, 
          floating=False, 
          scoring='accuracy',
          cv=10) # -1 means ALL CPU

svm_sbs.fit(X, Y, custom_feature_names=X.columns)


STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

KeyError: None

In [ ]:
selected_features = X.columns[list(svm_sbs.k_feature_idx_)]
x_lr = X_train[selected_features]
svm_clf = svm.SVC()
svm_clf.fit(x_lr, y_train)
print(cross_val_score(svm_clf, x_lr, y_train, cv=10).mean())

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Sequential Backward Selection
lr_sbs = SFS(LogisticRegression(), 
          k_features=(1, 10), 
          forward=False, 
          floating=False, 
          scoring='accuracy',
          cv=10) # -1 means ALL CPU

lr_sbs.fit(X, Y, custom_feature_names=X.columns)

selected_features = X.columns[list(lr_sbs.k_feature_idx_)]
x_lr = X_train[selected_features]
lr = LogisticRegression()
lr.fit(x_lr, y_train)
print(cross_val_score(lr, x_lr, y_train, cv=10).mean())

### Random Forest

In [8]:
# Sequential Backward Selection
rfc_sbs = SFS(RandomForestClassifier(random_state=0), 
          k_features=(1, 10), 
          forward=False, 
          floating=False, 
          scoring='accuracy',
          cv=10) # -1 means ALL CPU

rfc_sbs.fit(X, Y, custom_feature_names=X.columns)

selected_features = X.columns[list(rfc_sbs.k_feature_idx_)]
x_lr = X_train[selected_features]
rfc_ = RandomForestClassifier(random_state=0)
rfc_.fit(x_lr, y_train)
print(cross_val_score(rfc_, x_lr, y_train, cv=10).mean())

0.9231578947368421


### Multilayer Perceptron

In [ ]:
# Sequential Backward Selection
mlp_sbs = SFS(MLPClassifier(random_state=1), 
          k_features=(1, 10), 
          forward=False, 
          floating=False, 
          scoring='accuracy',
          cv=10) # -1 means ALL CPU

mlp_sbs.fit(X, Y, custom_feature_names=X.columns)

selected_features = X.columns[list(mlp_sbs.k_feature_idx_)]
x_lr = X_train[selected_features]
mlp_ = MLPClassifier(random_state=1)
mlp_.fit(x_lr, y_train)
print(cross_val_score(mlp_, x_lr, y_train, cv=10).mean())